In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv("mushrooms.csv")


In [21]:
#### Soru1,Soru2,Soru3 ve Soru4'ün Random Forest kullanılarak Çözümleri ####
#Random Forest kullaranak sınıflandırma işlemi ve sonuç değerlerinin hesaplanması ve ardından Random Search ile hiperparametre optimizasyonu

### Random Forest kulanabilmek için tüm değerlerin sayısal olması gerekiyor bu nedenle encoding yapmalıyız
dff = df.copy() #daha rahat işlem yapabilmek için ana veri setini kopyaladık

categorical_columns = [col for col in dff.columns if dff[col].dtypes == "O"] #list comprehension kullanarak kategorik sütünları belirledik
categorical_columns = [col for col in dff.columns if col not in ['class']] #hedef değişken olduğu için class sınıfını çıkarıyoruz

def one_hot_encoder(dataframe, categorical_cols): #Encoding yapmak için "One Hot Encoding" yöntemini tercih ettim ve işlemi fonksiyon haline getirdim
  dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=True)
  return dataframe

dff = one_hot_encoder(dff,categorical_columns ) # Hedef değişken hariç tüm değişkenlere one hot encoding uyguladım

y = dff['class'] # hedef değişken
X = dff.drop('class', axis=1)


from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=20)

from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=46).fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

print("######################### Hiperparamatre Optimizasyonu Öncesi ##########################")
print(f"Accuracy:  {accuracy_score(y_pred, y_test):.2f}")
print(f"Precision: {precision_score(y_test, y_pred, average='macro'):.2f}")
print(f"Recall: {recall_score(y_test, y_pred, average='macro'):.2f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='micro'):.2f}")
print(f"Karışıklık Matrisi:\n", confusion_matrix(y_test, y_pred))

print("######################### Hiperparamatre Optimizasyonu Sonrası ##########################")

# Random Forest'a olacak toplam ağaçları belirleme işlemi.
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)] # List comprehension ile 10 ile 80 arasında eşit aralıklarda 10 sayı ürettik
# Ağaçtaki her düğümde (node) özellik sayısının karekökü kadar değer seçilecek
max_features = ['sqrt']
# Ağaçtaki derinlik
max_depth = [2,4]
# Ağaçtaki bir düğümü (node) bölmek için gereken minimum örnek sayısı
min_samples_split = [2, 5]
# Her yaprak düğümde (leaf node) bulunması gereken minimum örnek sayısı
min_samples_leaf = [1, 2]
# Her ağaç için eğitimin nasıl olacağını belirleme. True ise ağaç verilerden rastgele örnekler alaca
# False ise ağaç tüm veri setiyle eğitilecek
bootstrap = [True, False]


param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

from sklearn.model_selection import RandomizedSearchCV

# Randomized Search ile hiperparametre optimizasyonu yapacağız. Optimazyomu yapılacak model önceden tanımladığımız "rf_model"
# param_distributions ise önceden belirledeğimiz parametreler. Model bu parametreler üzerinde deneme yapacak
rf_RandomGrid = RandomizedSearchCV(estimator = rf_model, param_distributions = param_grid, cv = 10, verbose=2, n_jobs = 4)

rf_RandomGrid.fit(X_train, y_train)

best_parameters = rf_RandomGrid.best_params_
print("Bulunan En İyi Parametler", best_parameters)

# Optimal parametreleri kullanarak yeniden modeli eğitiyoruz
rf_model = RandomForestClassifier(n_estimators=72,min_samples_split=2, min_samples_leaf=1, max_features='sqrt', max_depth=4, bootstrap=False).fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
print(f"Optimizasyon Sonrası Accuracy:  {accuracy_score(y_pred, y_test):.2f}")
print(f"Optimizasyon Sonrası Precision: {precision_score(y_test, y_pred, average='macro'):.2f}")
print(f"Optimizasyon Sonrası Recall: {recall_score(y_test, y_pred, average='macro'):.2f}")
print(f"Optimizasyon Sonrası F1 Score: {f1_score(y_test, y_pred, average='micro'):.2f}")
print(f"Optimizasyon Sonrası Karışıklık Matrisi:\n", confusion_matrix(y_test, y_pred))



######################### Hiperparamatre Optimizasyonu Öncesi ##########################
Accuracy:  0.93
Precision: 0.94
Recall: 0.93
F1 Score: 0.93
Karışıklık Matrisi:
 [[1266    0]
 [ 167 1005]]
######################### Hiperparamatre Optimizasyonu Sonrası ##########################
Fitting 10 folds for each of 10 candidates, totalling 100 fits
Bulunan En İyi Parametler {'n_estimators': 80, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 4, 'bootstrap': False}
Optimizasyon Sonrası Accuracy:  0.99
Optimizasyon Sonrası Precision: 0.99
Optimizasyon Sonrası Recall: 0.99
Optimizasyon Sonrası F1 Score: 0.99
Optimizasyon Sonrası Karışıklık Matrisi:
 [[1266    0]
 [  28 1144]]


In [27]:
#Support Vector Machine "Linear Kernel" kullaranak sınıflandırma işlemi ve sonuç değerlerinin hesaplanması ardından Random Search ile hiperparamatre optimizasyonu yapılması
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold


df = pd.read_csv("mushrooms.csv")

df["class"].value_counts() # class yani hedef değişkenimizde 2 sınıf var e ve p

y = dff['class']
X = dff.drop('class', axis=1)

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=109)


from sklearn import svm


clf = svm.SVC(kernel='linear', random_state=47, gamma='scale', C=0.1) # Support Vector Machine Linear Kernel Kullanımı


clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("######################### Hiperparamatre Optimizasyonu Öncesi ##########################")
print(f"Accuracy:  {accuracy_score(y_pred, y_test):.2f}")
print(f"Precision: {precision_score(y_test, y_pred, average='macro'):.2f}")
print(f"Recall: {recall_score(y_test, y_pred, average='macro'):.2f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='micro'):.2f}")
print(f"Karışıklık Matrisi:\n", confusion_matrix(y_test, y_pred))

print("######################### Hiperparamatre Optimizasyonu Sonrası ##########################")

parameter_grid = {
    "C" :  np.logspace(-10, 10, 20), #regülayonu ayarlamak için 10 üzeri -10 ile 10 üzeri 10 arasında 20 yane sayı ürettik
    "kernel" : ['linear', 'rbf'], #kullanılıcak çekirdek (kernel) belirlemek için.
    #linear verileri doğrusal bir şekilde ayırır, rbf (Radial Basis Function ) daha karmaşık veriler için
    "gamma" : np.logspace(-10, 10, 20) # gamma kernel fonksiyonların etkisini kontrol etmek için kullanılır.
    # daha yüksek gamma değeri modelin veriye daha fazla odaklanmasını sağlar
}
scoring = ['accuracy'] #modelin performansını ölçecek metrikleri belirleme işlemi

#modeli eğitim ve test setlerine ayırmak için cross validation kullandık
k_fold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

random_search_svm = RandomizedSearchCV(estimator = clf, param_distributions = parameter_grid,scoring=scoring, refit='accuracy', n_jobs=-1, cv = k_fold, verbose=0, n_iter = 100)

random_search_svm.fit(X_train, y_train)

best_parameters = random_search_svm.best_params_
print("Bulunan En İyi Parametler", best_parameters)

######################### Hiperparamatre Optimizasyonu Öncesi ##########################
Accuracy:  1.00
Precision: 1.00
Recall: 1.00
F1 Score: 1.00
Karışıklık Matrisi:
 [[1247    0]
 [   2 1189]]
######################### Hiperparamatre Optimizasyonu Sonrası ##########################
Bulunan En İyi Parametler {'kernel': 'linear', 'gamma': 1.1288378916846883e-09, 'C': 885866790.4100796}


In [31]:

# Support Vector Machine için Optimal parametreleri kullanarak yeniden modeli eğitiyoruz

clf = svm.SVC(kernel='linear', random_state=47, gamma=1.1288378916846883e-09, C=885866790.4100796)


clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("######################### Support Vector Machine Hiperparamatre Optimizasyonu Sonrası ##########################")
print(f"Optimizasyon Sonrası Accuracy:  {accuracy_score(y_pred, y_test):.2f}")
print(f"Optimizasyon Sonrası Precision: {precision_score(y_test, y_pred, average='macro'):.2f}")
print(f"Optimizasyon Sonrası Recall: {recall_score(y_test, y_pred, average='macro'):.2f}")
print(f"Optimizasyon Sonrası F1 Score: {f1_score(y_test, y_pred, average='micro'):.2f}")
print(f"Optimizasyon Sonrası Karışıklık Matrisi:\n", confusion_matrix(y_test, y_pred))

######################### Support Vector Machine Hiperparamatre Optimizasyonu Sonrası ##########################
Optimizasyon Sonrası Accuracy:  1.00
Optimizasyon Sonrası Precision: 1.00
Optimizasyon Sonrası Recall: 1.00
Optimizasyon Sonrası F1 Score: 1.00
Optimizasyon Sonrası Karışıklık Matrisi:
 [[1247    0]
 [   0 1191]]


In [38]:
#Gaussian Naive Bayes kullanılarak sınıflandırma işlemi ve sonuç değerlerinin hesaplanması arından Grid Search ile hiperparametre optimizasyonu yapılması

df = pd.read_csv("mushrooms.csv")

categorical_columns = [col for col in df.columns if df[col].dtypes == "O"] #list comprehension kullanarak kategorik sütünları belirledik
categorical_columns = [col for col in df.columns if col not in ['class']]

df = one_hot_encoder(df,categorical_columns )

y = df['class']
X = df.drop('class', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=125)

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

model = GaussianNB()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("######################### Hiperparamatre Optimizasyonu Öncesi ##########################")
print(f"Accuracy:  {accuracy_score(y_pred, y_test):.2f}")
print(f"Precision: {precision_score(y_test, y_pred, average='macro'):.2f}")
print(f"Recall: {recall_score(y_test, y_pred, average='macro'):.2f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='micro'):.2f}")
print(f"Karışıklık Matrisi:\n", confusion_matrix(y_test, y_pred))

print("######################### Hiperparamatre Optimizasyonu Sonrası ##########################")

from sklearn.model_selection import RepeatedStratifiedKFold

cv_method = RepeatedStratifiedKFold(n_splits=5,
                                    n_repeats=3,
                                    random_state=999)


from sklearn.preprocessing import PowerTransformer

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
# Gaussian Naive Bayes sadece tek bir tane argüman alır. "var_smoothing" = Hesaplama kararlılığı için varyanslara eklenen tüm özelliklerin en büyük varyansının kısmı.
# Yani varyans (farklılık/varience) hesaplamalardan sıfır elde edilmemesi için varyans değerine etklenen küçük bir miktardır
# farklılıklar sıfırdan büyük olursa model daha kararlı tahminler yapabilir.

from sklearn.model_selection import GridSearchCV

gs_NB = GridSearchCV(estimator=model,
                     param_grid=params_NB,
                     cv=cv_method,
                     verbose=1,
                     scoring='accuracy')

Data_transformed = PowerTransformer().fit_transform(X_test)
# veri setindeki dağılımını Gauss dağılımına daha da yaklaştırmak için PowerTransformer ile X_test verisini değiştirdik.

gs_NB.fit(Data_transformed, y_test) #değiştirilen veriler ile GridSearch'u eğittik

best_parameter_for_gaussian = gs_NB.best_params_
print("Bulunan En İyi Paramete", best_parameters)

# Gaussian Naive Bayes için Optimal parametreleri kullanarak yeniden modeli eğitiyoruz

model = GaussianNB(var_smoothing=best_parameter_for_gaussian['var_smoothing'])

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(f"Optimizasyon Sonrası Accuracy:  {accuracy_score(y_pred, y_test):.2f}")
print(f"Optimizasyon Sonrası Precision: {precision_score(y_test, y_pred, average='macro'):.2f}")
print(f"Optimizasyon Sonrası Recall: {recall_score(y_test, y_pred, average='macro'):.2f}")
print(f"Optimizasyon Sonrası F1 Score: {f1_score(y_test, y_pred, average='micro'):.2f}")
print(f"Optimizasyon Sonrası Karışıklık Matrisi:\n", confusion_matrix(y_test, y_pred))


######################### Hiperparamatre Optimizasyonu Öncesi ##########################
Accuracy:  0.94
Precision: 0.94
Recall: 0.94
F1 Score: 0.94
Karışıklık Matrisi:
 [[1255  159]
 [   0 1267]]
######################### Hiperparamatre Optimizasyonu Sonrası ##########################
Fitting 15 folds for each of 100 candidates, totalling 1500 fits
Bulunan En İyi Paramete {'var_smoothing': 0.012328467394420659}
Optimizasyon Sonrası Accuracy:  0.99
Optimizasyon Sonrası Precision: 0.99
Optimizasyon Sonrası Recall: 0.99
Optimizasyon Sonrası F1 Score: 0.99
Optimizasyon Sonrası Karışıklık Matrisi:
 [[1396   18]
 [   0 1267]]


# Soru 5  Çözümü
**1)** Elimizdeki veri setini RandomForestClassifier ile eğittikten sonra aldığım sonuçlar bu şekildeydi:
* Accuracy:  0.93
* Precision: 0.94
* Recall: 0.93
* F1 Score: 0.93

Ardından modele Random Search ile hiperparametre optimizasyonu yapınca modelimin başarı oranı arttı ve bu değerleri elde ettim:
  * Optimizasyon Sonrası Accuracy:  0.99
  * Optimizasyon Sonrası Precision: 0.99
  * Optimizasyon Sonrası Recall: 0.99
  * Optimizasyon Sonrası F1 Score: 0.99

**2)** Sınıflandırma için Support Vector Machine kullandığımda değerlendirme metriklerimin değerleri hepsi "1 "şeklinde geldi. Bu overfitting olabileceği anlamına geliyor. Daha sonra Random Search ile hipermarametre optimizasyonu yaptığımda sonuç yine aynı kaldı ve tüm metrikler "1" sonucunu verdi.
Hiperparametre optimiazsyon işlemi en uzun burada gerçekleşti, işlem yaklaşık 8dk sürdü

**3)**Gaussian Naive Bayes sınıfandırma kullandığımda ilk öncelikle bu sonuçları aldım:
  * Accuracy:  0.94
  * Precision: 0.94
  * Recall: 0.94
  * F1 Score: 0.94
  
Ardından Grid Search kullanarak hiperparametre optimizasyonu yaptığımda model daha iyi sonuç verdi ve metriklerin değerleri arttı:
  * Optimizasyon Sonrası Accuracy:  0.99
  * Optimizasyon Sonrası Precision: 0.99
  * Optimizasyon Sonrası Recall: 0.99
  * Optimizasyon Sonrası F1 Score: 0.99
